# WNTR Geospatial Tutorial
The following tutorial illustrates the use of the `wntr.gis` module to use geospatial data in resilience analysis.  The objective of this tutorial is to 1) quantify water service disruptions that could occur from pipes damaged in landslides and 2) identify the social vulnerability of populations impacted by the service disruptions.

To simplify the tutorials, it is assumed that pipes within a 1000 ft buffer of each landslide susceptible region are damaged in that landslide.
This assumption could be replaced with detailed landslide analysis that includes slope, soil type, weather conditions, and pipe material.
Social vulnerability data could also be replaced by datasets that describe other attributes of the population and critical services.

## Imports
Import WNTR and additional Python packages that are needed for the tutorial
- Geopandas is used to load geospatial data
- Shapely is used to define a region of interest to crop data
- Matplotlib is used to create subplots

In [ ]:
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pylab as plt
import wntr

## Units
WNTR uses SI (International System) units (length in meters, time in seconds, mass in kilograms), **with the exception of the landslide buffer which is in feet to match the coordinate reference system of the geospatial data**.  See https://usepa.github.io/WNTR/units.html for more details on WNTR units.

In [ ]:
# The following line defines coordinates used to zoom in on network graphics
zoom_coords = [(5.75e6, 5.79e6), (3.82e6, 3.85e6)] 

# Water Network Model
The following section creates a `WaterNetworkModel` object from an EPANET INP file and converts the model to GeoDataFrames for use in geospatial analysis.

## Create a WaterNetworkModel from an EPANET INP file
The water distribution network model used in this tutorial was downloaded from the [UKnowledge Water Distribution Systems Research Database](https://uknowledge.uky.edu/wdsrd/). KY10 was selected for the analysis. The following section creates a `WaterNetworkModel` from an EPANET INP file and computes some general attributes of the model.

*Citation: Hoagland, Steven, "10 KY 10" (2016). Kentucky Dataset. 12. https://uknowledge.uky.edu/wdst/12. Accessed on 4/4/2024.*

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
inp_file = '../networks/ky10.inp'
wn = wntr.network.WaterNetworkModel(inp_file)

In [ ]:
# Print a basic description of the model. 
# The level can be 0, 1, or 2 and defines the level of detail included in the description.
wn.describe(level=1)

In [ ]:
# Compute total pipe length
length = wn.query_link_attribute('length')
total_length = length.sum() # m
print('Total pipe length =', total_length, 'm, =', total_length*3.28084, 'ft')

In [ ]:
# Compute average expected demand per day 
average_expected_demand = wntr.metrics.average_expected_demand(wn) # m^3/s
average_volume_per_day = average_expected_demand*(24*3600) # m^3
total_water_use = average_volume_per_day.sum() # m^3
print('Total water use =', total_water_use, 'm^3, =', total_water_use*264.172/1e6, 'million gallons')

In [ ]:
# Estimate population using the default average volume of water consumed per capita per day of 200 gallons/day
population = wntr.metrics.population(wn) 
total_population = population.sum()
print('Total population =', total_population)

In [ ]:
# Create a basic network graphic, showing junction elevation
# Note, the remaining graphics in this tutorial are created from the geospatial data directly, rather than the `plot_network` function.
# The `plot_network` function currently does not include vertices.
ax = wntr.graphics.plot_network(wn, node_attribute='elevation', node_range=(175, 300), title='ky10 elevation')

## Convert the WaterNetworkModel to GIS data
The `WaterNetworkModel` is converted to a collection of GIS compatible GeoDataFrames and the coordinate reference system (CRS) is set to **EPSG:3089 (NAD83 / Kentucky Single Zone (ftUS)**, see https://epsg.io/3089 for more details).  Data for junctions, tanks, reservoirs, pipes, pumps, and valves are stored in separate GeoDataFrames.

In [ ]:
# Convert the WaterNetworkModel to GIS data and set the CRS
wn_gis = wn.to_gis()
wn_gis.junctions.head()
#wn_gis.tanks.head()
#wn_gis.reservoirs.head()
#wn_gis.pipes.head()
#wn_gis.pumps.head()
#wn_gis.tanks.head()


In [ ]:
# Set the CRS to EPSG:3089 (NAD83 / Kentucky Single Zone (ftUS))
crs = 'EPSG:3089'
wn_gis.set_crs(crs)

In [ ]:
# Use the GIS data to create a figure of the network
fig, ax = plt.subplots(figsize=(5,5))
ax = wn_gis.pipes.plot(column='diameter', linewidth=1, label='pipes', alpha=0.8, ax=ax, zorder=1)
ax = wn_gis.reservoirs.plot(color='k', marker='s', markersize=60, label='reservoirs', ax=ax)
ax = wn_gis.tanks.plot(color='r', markersize=20, label='tanks', ax=ax)
ax = wn_gis.pumps.centroid.plot(color='b', markersize=20, label='pumps', ax=ax)
ax = wn_gis.valves.centroid.plot(color='c', markersize=20, label='valves', ax=ax)
tmp = ax.axis('off')
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
#tmp = ax.set_xlim(zoom_coords[0])
#tmp = ax.set_ylim(zoom_coords[1])
tmp = plt.legend()

## Save the GIS data to GeoJSON or Shapefile files
The GIS data can be written to GeoJSON files or Shapefile files.  One file is created for junctions, tanks, reservoirs, pipes, pumps, and valves (ky10_junctions.geojson, ky10_tanks.geojson, etc.).  The GeoJSON or Shapefile files can be loaded into GIS software platforms for further analysis. **Note that controls, patterns, curves, and options are not included in the GIS formatted data files.** 

In [ ]:
# Store the WaterNetworkModel as a collection of GeoJSON files
wn_gis.write_geojson('ky10')

# External GIS Data
The external data used in this tutorial includes landslide inventory and social vulnerability data.

## Load landslide GIS data
The landslide data used in this tutorial was downloaded from the [UKnowledge Kentucky Geological Survey Research Data](https://uknowledge.uky.edu/kgs_data/).  The Kentucky Geological Survey Landslide Inventory from March 2023 was selected for the analysis.  The data contains locations of known landslides and areas susceptible to debris flows, derived from aerial photography. 

*Citation: Crawford, M.M., 2023. Kentucky Geological Survey landslide inventory [2023-03]: Kentucky Geological Survey Research Data, https://uknowledge.uky.edu/kgs_data/7/, Accessed on 4/4/2024.*

In [ ]:
# To reduce the file size checked into the WNTR repository, the following code was run on the raw data file.
# The region of interest (ROI) includes a 5000 ft buffer surrounding all pipes. The function `box` was imported from shapely.
"""
bounds = wn_gis.pipes.total_bounds # total_bounds returns the upper and lower bounds on x and y
geom = box(*bounds)
ROI = geom.buffer(5000) # feet

landslide_file = '../data/KGS_Landslide_Inventory_exp.gdb'
landslide_data = gpd.read_file(landslide_file, driver="FileGDB", layer='Areas_derived_from_aerial_photography')
print(landslide_data.crs)
landslide_data = landslide_data.clip(ROI)
landslide_data.to_file("../data/ky10_landslide_data.geojson", index=True, driver='GeoJSON')
"""

In [ ]:
# Load the landslide data from file and print the CRS to ensure it is in EPSG:3089.  
# The methods `to_crs` and `set_crs` can be used to change coordinate reference systems if needed.
landslide_file = '../data/ky10_landslide_data.geojson'
landslide_data = gpd.read_file(landslide_file).set_index('index') 
print(landslide_data.crs)

landslide_data.head()

In [ ]:
# Plot the landslide data along with pipes
ax = landslide_data.plot(color='red', label='Landslide data')
ax = wn_gis.pipes.plot(color='black', linewidth=1, ax=ax)
ax.set_title('Landslide and pipe data')
tmp = ax.axis('off')
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
tmp = ax.set_xlim(zoom_coords[0])
tmp = ax.set_ylim(zoom_coords[1])

## Load Social Vulnerability Index (SVI) GIS data
The social vulnerability data used in this tutorial was downloaded from the [Centers for Disease Control and Prevention/Agency for Toxic Substances and Disease Registry](https://www.atsdr.cdc.gov/placeandhealth/svi/index.html). The data contains census and social vulnerability metrics for each census tract. 

The quantity of interest used in this analysis is `RPL_THEMES` which ranks vulnerability across socioeconomic status, household characteristics, racial and ethnic minority status, and housing type and transportation.  The value ranges between 0 and 1, where higher values are associated with higher vulnerability.

*Citation: Centers for Disease Control and Prevention/Agency for Toxic Substances and Disease Registry/Geospatial Research, Analysis, and Services Program. CDC/ATSDR Social Vulnerability Index 2020 Database Kentucky. https://www.atsdr.cdc.gov/placeandhealth/svi/data_documentation_download.html. Accessed on 4/4/2024.*

In [ ]:
# To reduce the file size checked into the WNTR repository, the following code was run on the raw data file. 
# The region of interest (ROI) was defined above.
"""
svi_file = '../data/SVI2020_KENTUCKY_tract.gdb'
svi_data = gpd.read_file(svi_file, driver="FileGDB", layer='SVI2020_KENTUCKY_tract')
print(svi_data.crs)
svi_data.to_crs(crs, inplace=True)
svi_data = svi_data.clip(ROI)
svi_data.to_file("../data/ky10_svi_data.geojson", index=True, driver='GeoJSON')
"""

In [ ]:
# Load the SVI data from file and print the CRS to ensure it is in EPSG:3089.  
# The methods `to_crs` and `set_crs` can be used to change coordinate reference systems if needed.
svi_file = '../data/ky10_svi_data.geojson'
svi_data = gpd.read_file(svi_file).set_index('index') 
print(svi_data.crs)

svi_data.head()

In [ ]:
# Plot SVI data and pipes (higher values of SVI are associated with higher vulnerability)
ax = svi_data.plot(column='RPL_THEMES', label='SVI data', cmap='RdYlGn_r', vmin=0, vmax=1, legend=True)
ax = wn_gis.pipes.plot(color='black', linewidth=1, ax=ax)
ax.set_title('SVI and pipe data')
tmp = ax.axis('off')
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
tmp = ax.set_xlim(zoom_coords[0])
tmp = ax.set_ylim(zoom_coords[1])

## Expand the size of each landslide using a buffer
Each landslide is extended to include the surrounding 1000 ft, to create a region that might be impacted by an individual landslide.  The distance unit for buffering matches the distance unit of the CRS (ft).
This assumption could be replaced with detailed landslide analysis that includes slope, soil type, weather conditions, and pipe material. 

In [ ]:
# Create a GeoDataFrame to hold information used in landslide scenarios (initially copied from landslide_data)
# Buffer each landslide polygon by 1000 ft
landslide_scenarios = landslide_data.copy()
landslide_scenarios['geometry'] = landslide_data.buffer(1000)

In [ ]:
# Add a prefix to the landslide scenario index to indicate the scenario name
landslide_scenarios.index = 'LS-' + landslide_scenarios.index.astype(str)

In [ ]:
# Plot the landslide data, region included in each landslide scenario, and pipes
ax = landslide_scenarios.plot(color='gray', alpha=0.5)
ax = landslide_data.plot(color='red', label='Landslide data', ax=ax)
ax = wn_gis.pipes.plot(color='black', linewidth=1, ax=ax)
ax.set_title('Landslide scenario and pipe data')
tmp = ax.axis('off')
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
tmp = ax.set_xlim(zoom_coords[0])
tmp = ax.set_ylim(zoom_coords[1])

# Geospatial Intersects
In this section, landslide scenario and SVI data are interested with pipes and junctions in the `WaterNetworkModel`.

## Identify pipes that intersect each landslide
Landslide polygons are intersected with pipes to obtain a list of pipes that intersect each landslide.  <font color='red'>This information is used to to define the pipes that are closed in each landslide scenario.</font> The pipe attribute `length` is also included in the intersection to gather statistics on the pipe length that intersects each landslide.  

In [ ]:
# Use the intersect function to determine pipes and pipe length that intersects each landslide
A = landslide_scenarios
B = wn_gis.pipes
B_value = 'length'
landslide_intersect = wntr.gis.intersect(A, B, B_value)

# Print results in order of descending total pipe length
landslide_intersect.sort_values('sum', ascending=False).head()

In [ ]:
# Add the intersection results to the landslide scenario data
landslide_scenarios[['intersections', 'n', 'total pipe length']] = landslide_intersect[['intersections', 'n', 'sum']]

# Print results in order of descending total pipe length
landslide_scenarios.sort_values('total pipe length', ascending=False).head()

In [ ]:
# Plot intersection results
fig, axes = plt.subplots(1,2, figsize=(15,5))

wn_gis.pipes.plot(color='gray', linewidth=1, ax=axes[0])
landslide_scenarios.plot(column='n', vmax=10, legend=True, ax=axes[0])
tmp = axes[0].set_title('Number of pipes that intersect each landslide')
tmp = axes[0].axis('off')
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
tmp = axes[0].set_xlim(zoom_coords[0])
tmp = axes[0].set_ylim(zoom_coords[1])

wn_gis.pipes.plot(color='gray', linewidth=1, ax=axes[1])
landslide_scenarios.plot(column='total pipe length', vmax=10000, legend=True, ax=axes[1])
tmp = axes[1].set_title('Length of pipe that intersect each landslide')
tmp = axes[1].axis('off')
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
tmp = axes[1].set_xlim(zoom_coords[0])
tmp = axes[1].set_ylim(zoom_coords[1])

## Identify landslides that intersect each pipe
Pipes are intersected with landslides to obtain a list of landslides that intersect each pipe. The landslide attribute `Confidence_Ranking` is also included in the intersection to gather statistics on landslide confidence for each pipe.  <font color='red'>While this information is not used in the analysis below, this type of information could be used to inform uncertainty or probability of damage.</font>

**Note that `Confidence_Ranking` has a value of 3 ("Landslide likely at or near the specified location") for each landslide in region of interest. Since the values are uniform in this dataset, the intersected sum, min, max, and mean are all the same value.** More information on Confidence ranking can be found at https://kgs.uky.edu/kgsmap/helpfiles/landslide_help.shtm.  

In [ ]:
# Use the intersect function to determine landslides and landslide confidence ranking that intersects each pipe
A = wn_gis.pipes
B = landslide_scenarios
B_value = 'Confidence_Ranking'
pipe_intersect = wntr.gis.intersect(A, B, B_value)

# Print results in order of descending number of intersections.
pipe_intersect.sort_values('n', ascending=False).head()

In [ ]:
# Add the intersection results to the GIS pipe data
wn_gis.pipes[['intersections', 'n', 'Confidence_Ranking']] = pipe_intersect[['intersections', 'n', 'mean']]

# Print results in order of descending number of intersections
wn_gis.pipes.sort_values('n', ascending=False).head()

In [ ]:
# Plot intersection results
ax = wn_gis.pipes.plot(color='gray', linewidth=1, zorder=1)
wn_gis.pipes[wn_gis.pipes['n'] > 0].plot(column='n', vmax=20, legend=True, ax=ax)
tmp = ax.set_title('Number of landslide scenarios that intersect each pipe')
tmp = ax.axis('off')
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
tmp = ax.set_xlim(zoom_coords[0])
tmp = ax.set_ylim(zoom_coords[1])

## Assign social vulnerability to each junction
Junctions are intersected with SVI to determine the social vulnerability of the population at each junction.  The SVI data attribute `RPL_THEMES` is included in the intersection. <font color='red'>This information is used to determine the social vulnerability of individuals that experience water service disruptions.</font>

The SVI data column `RPL_THEMES` ranks vulnerability across socioeconomic status, household characteristics, racial and ethnic minority status, and housing type and transportation. The value ranges between 0 and 1, where higher values are associated with higher vulnerability.

**Note that since each junction only intersects one census tract, the SVI sum, min, max, and mean are all the same value.**

In [ ]:
# Use the intersect function to determine SVI of each junction.  
A = wn_gis.junctions
B = svi_data
B_value = 'RPL_THEMES'
junction_svi = wntr.gis.intersect(A, B, B_value)

junction_svi.head()

In [ ]:
# Add the intersection results (SVI value) to the GIS junction data
wn_gis.junctions['RPL_THEMES'] = junction_svi['mean']

In [ ]:
# Plot SVI for each census tract and SVI assigned to each junction
fig, axes = plt.subplots(1,2, figsize=(15,5))

svi_data.plot(column='RPL_THEMES', label='SVI data', vmin=0, vmax=1, legend=True, ax=axes[0])
wn_gis.pipes.plot(color='black', linewidth=1, ax=axes[0])
tmp = axes[0].set_title('Census tract SVI and pipe data')
tmp = axes[0].axis('off')
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
tmp = axes[0].set_xlim(zoom_coords[0])
tmp = axes[0].set_ylim(zoom_coords[1])

wn_gis.pipes.plot(color='gray', linewidth=1, ax=axes[1])
wn_gis.junctions.plot(column='RPL_THEMES', vmin=0, vmax=1, legend=True, ax=axes[1])
tmp = axes[1].set_title('SVI value at each junction')
tmp = axes[1].axis('off')
# Comment/uncomment the following 2 lines to change the zoom on the network graphic
tmp = axes[1].set_xlim(zoom_coords[0])
tmp = axes[1].set_ylim(zoom_coords[1])

# Hydraulic Simulations
The following section runs hydraulic simulations for the baseline (no landslide) and landslide scenarios. A subset of landslide scenarios is run to simply the tutorial.  Simulation results are stored for later analysis.

In [ ]:
# Create a function to setup the WaterNetworkModel for hydraulic simulations
def model_setup(inp_file):
    wn = wntr.network.WaterNetworkModel(inp_file)
    wn.options.hydraulic.demand_model = 'PDD'
    wn.options.hydraulic.required_pressure = 20 # m
    wn.options.hydraulic.minimum_pressure  = 0 # m
    wn.options.time.duration = 48*3600 # s (48 hour simulation)
    return wn

## Run baseline simulation

In [ ]:
# Run a baseline simulation, with no landslide or damage.  
wn = model_setup(inp_file)
sim = wntr.sim.EpanetSimulator(wn)
baseline_results = sim.run_sim()

In [ ]:
# View a subset of the simulation results
baseline_results.node['pressure'].head()

## Run landslide scenarios
Landslide scenarios are downselected by identifying the set of landslides that impact a unique set of pipes.  Scenarios are further downselected to 6 scenarios to simplify the tutorial.  A hydraulic simulation is run for each landslide scenario, where pipes that intersect the landslide are closed for 48 hours.  Results from each scenario are stored for later analysis.

In [ ]:
# Remove scenarios with no intersecting pipes
landslide_scenarios = landslide_scenarios[landslide_scenarios['n'] > 0]
landslide_scenarios = landslide_scenarios[~landslide_scenarios['n'].isna()]

# Downselect landslide scenarios that impact a unique set of pipes
duplicated_intersections = landslide_scenarios['intersections'].astype(str).duplicated()
landslide_scenarios = landslide_scenarios.loc[~duplicated_intersections, :]

print('Number of unique landslide scenarios', landslide_scenarios.shape[0])
landslide_scenarios.head()

In [ ]:
# Further downselect the landslide scenarios for demonstration purposes. Choose one of the following 4 options.
# Option 1. 6 scenarios that illustrate a wide range of impact
landslide_scenarios_downselect = landslide_scenarios.loc[['LS-4495', 'LS-7003', 'LS-7111', 'LS-5086', 'LS-6966', 'LS-7058'],:] 

# Option 2. 6 scenarios with the highest intersecting pipe length
#landslide_scenarios_downselect = landslide_scenarios.sort_values('total pipe length', ascending=False).iloc[0:6,:]

# Option 3. 6 scenarios with the highest number of intersecting pipes
#landslide_scenarios_downselect = landslide_scenarios.sort_values('n', ascending=False).iloc[0:6,:]

# Option 4. Random selection of 6 scenarios
#landslide_scenarios_downselect = landslide_scenarios.sample(n=6, random_state=1234)

landslide_scenarios_downselect

In [ ]:
# Plot the location of landslides used in the analysis
ax = landslide_scenarios_downselect.plot(color='blue')
wn_gis.pipes.plot(color='gray', linewidth=1, ax=ax)
tmp = ax.set_title('Landslide scenarios')
tmp = ax.axis('off')

In [ ]:
# Run a hydraulic simulation for each landslide scenario, store results in a dictionary
# Each scenario closes all pipes that intersect the landslide for the 48 hour simulation
results = {}
for i, scenario in landslide_scenarios_downselect.iterrows():
    wn = model_setup(inp_file)
    for pipe_i in scenario['intersections']:
        pipe_object = wn.get_link(pipe_i)
        pipe_object.initial_status = 'CLOSED'
    sim = wntr.sim.EpanetSimulator(wn)
    results[i] = sim.run_sim()

# Analysis Results
The following section computes and plots analysis results, including water service availability (WSA) and the social vulnerability index (SVI) of impacted junctions for each scenario.

## Water Service Availability (WSA)
Water service availability (WSA) is the ratio of delivered demand to the expected demand.  WSA is computed for each junction (alternatively, WSA can be computed for each timestep, or for each junction and timestep).  A value below 1 indicates that expected demand it me, while a value of 0 indicates that the expected demand is not met. 

**Note that WSA can be > 1 and < 0 due to numerical differences in expected and actual demand. For certain types of analysis, WSA should be truncated to values between 0 and 1.**

In [ ]:
# Compute expected demand for each junction and timestep
expected_demand = wntr.metrics.expected_demand(wn)

expected_demand.head()

In [ ]:
# Compute total expected demand at each junction (axis 0 is the time index)
expected_demand_j = expected_demand.sum(axis=0)

expected_demand_j

In [ ]:
# Compute baseline WSA for each junction
demand_baseline = baseline_results.node['demand'].loc[:,wn.junction_name_list]
demand_baseline_j = demand_baseline.sum(axis=0) # total demand at each junction
wsa_baseline_j = wntr.metrics.water_service_availability(expected_demand_j, demand_baseline_j)

wsa_baseline_j.head()

In [ ]:
# Add WSA from the base simulation to the junction GIS data
wn_gis.junctions['baseline'] = wsa_baseline_j

In [ ]:
# Plot WSA from the base simulation
ax = wn_gis.pipes.plot(color='black', linewidth=1)
ax = wn_gis.junctions.plot(column='baseline', cmap='RdYlGn', vmin=0, vmax=1, legend=True, ax=ax)
tmp = ax.set_title('Baseline WSA')
tmp = ax.axis('off')

In [ ]:
# Compute WSA associated with each landslide scenarios
for i, scenario in landslide_scenarios_downselect.iterrows():
    demand = results[i].node['demand'].loc[:,wn.junction_name_list]
    demand_j = demand.sum(axis=0) # total demand at each junction
    wsa_j = wntr.metrics.water_service_availability(expected_demand_j, demand_j)
    
    # Add WSA to the junction GIS data
    wn_gis.junctions[i] = wsa_j
    print(i, len(scenario['intersections']), wsa_j.mean())

In [ ]:
# Extract WSA for each scenario 
wsa_results = wn_gis.junctions[landslide_scenarios_downselect.index]

wsa_results.head()

In [ ]:
# Plot distribution of WSA for each scenario. Note that WSA can be > 1 and < 0 due to numerical differences in expected and actual demand. 
# For certain types of analysis, the WSA should be truncated to values between 0 and 1.
ax = wsa_results.boxplot()
tmp = ax.set_ylim(-0.25, 1.25)
tmp = ax.set_ylabel('WSA')
tmp = ax.set_title('Distribution of WSA for each scenario')

In [ ]:
# Plot WSA for each scenario
fig, axes = plt.subplots(2,3, figsize=(15,10))
axes = axes.flatten()

for i, scenario in enumerate(wsa_results.columns):
    wn_gis.pipes.plot(color='gray', linewidth=1, ax=axes[i]) # pipes
    wn_gis.junctions.plot(column=scenario, cmap='RdYlGn', vmin=0, vmax=1, legend=True, ax=axes[i]) # junction wsa
    tmp = axes[i].set_title('WSA '+scenario)
    tmp = axes[i].axis('off')
    if i >= 6: # axes is defined to have 6 subplots
        break

## SVI of impacted junctions
In this analysis, impacted junctions are defined as junctions where WSA falls below 0.5 (50% of the expected water was received) at any time during the simulation. Other criteria could also be used to defined impact.

In [ ]:
# Extract junctions that are impacted by WSA < 0.5 for each scenario
impacted_junctions = {}
for scenario in wsa_results.columns:
    filter = wsa_results[scenario] < 0.5
    impacted_junctions[scenario] = wsa_results.index[filter]

impacted_junctions

In [ ]:
# Plot SVI of impacted junctions for each scenario
fig, axes = plt.subplots(2,3, figsize=(15,10))
axes = axes.flatten()

for i, scenario in enumerate(wsa_results.columns):
    j = impacted_junctions[scenario]
    wn_gis.pipes.plot(color='gray', linewidth=1, alpha=0.5, ax=axes[i]) # pipes
    if len(j) > 0:
        wn_gis.junctions.loc[j,:].plot(column='RPL_THEMES', cmap='RdYlGn_r', vmin=0, vmax=1, legend=True, ax=axes[i]) # junction wsa
    tmp = axes[i].set_title('SVI '+scenario)
    tmp = axes[i].axis('off')
    if i >= 6: # axes is defined to have 6 subplots
        break

## Save analysis results to GIS files
The analysis above stored WSA results for each scenario to the `wn_gis` object, which can be saved to GIS formatted files and loaded into GIS software platforms for further analysis.  **Note that lists (such as the information stored in 'intersections') is not JSON serializable and must first be removed.**

In [ ]:
del wn_gis.pipes['intersections']
wn_gis.write_geojson('ky10_analysis_results')